In [1]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
from scipy.spatial.distance import euclidean # add change

model = YOLO('yolov8n.pt')
car_class_id = 2  # Class ID for cars
bus_class_id = 5  # Class ID for buses
truck_class_id = 7  # Class ID for trucks
confidence_threshold = 0.5

In [2]:
cap=cv2.VideoCapture('highway.mp4')

Area_of_Interest = [(200,210), (410,210),(360,330),(1,320)] # top-left -> top-right -> bottom-right -> bottom-left

avg_car_length = 4039.19 # in milimeters (exact value = 4,039.188636363636)
avg_car_width = 1572.89 # in milimeters (exact value = 1,572.890909090909)

#add speed
fps = cap.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the video

#add change
previous_positions = {}
next_object_id = 0

#add change (Assigns a unique ID to a new object based on its center position)
def assign_id_to_new_object(cx, cy, previous_positions, threshold=50):
    global next_object_id

    for obj_id, (prev_x, prev_y) in previous_positions.items():
        if euclidean((cx, cy), (prev_x, prev_y)) < threshold:
            return obj_id  # Object is already tracked

    # Assign new ID to an untracked object
    obj_id = next_object_id
    next_object_id += 1
    return obj_id

while True:
    cap_check, frame = cap.read()
    if not cap_check:
        break
    
    frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
    cv2.imwrite('saved_image.jpg', frame)
    
    height, width = frame.shape[:2]
    print(f'Resized Frame Size: Width = {width}, Height = {height}')
    # print(f'y2 = {int(y2)}, y1 = {int(y1)}, x2 = {int(x2)}, x1 = {int(x1)}')
    # print(f'Distance per pixel = {distance_per_pixel}')

    results = model(frame)
    detections = results[0].boxes

    current_positions = {} #add change

    for box in detections.data:

        x1, y1, x2, y2, conf, cls = box.tolist()
        cx = int(x1+x2) //2
        cy = int(y1+y2) //2

        if conf >= confidence_threshold and (cls == car_class_id or cls == truck_class_id or cls == bus_class_id):
                
            # return 1 if inside, 0 if on the edge, -1 if outside 
            result = cv2.pointPolygonTest(np.array(Area_of_Interest,np.int32),(int(cx),int(cy)),False) 

            if(result >= 0):
                # add speed
                object_id = assign_id_to_new_object(cx, cy, previous_positions) # add change
                current_positions[object_id] = (cx, cy)
    
                # Draw the bounding box
                cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                # Put class ID or name on the frame
                cv2.putText(frame, f'Conf: {conf:.2f}', (int(x1), int(y1) - 10), #ID: {int(cls)} 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                distance_per_pixel = (avg_car_width * 0.001) / int(y2-y1) # in meters/pixel

                #add speed
                if object_id in previous_positions: # add change

                    # Get previous position
                    prev_x, prev_y = previous_positions[object_id]

                    # Calculate the distance moved in pixels
                    pixel_distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)

                    # Convert pixel distance to real-world distance (in meters)
                    real_world_distance = pixel_distance * distance_per_pixel

                    # Calculate speed (distance/time), time = 1/fps
                    speed = real_world_distance * fps  # Speed in meters per second

                    # Display speed on the frame (convert to km/h by multiplying by 3.6)
                    cv2.putText(frame, f'Speed: {speed * 3.6:.2f} km/h', (int(x1), int(y2) + 20), # Speed
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 200, 255), 2)
                    cv2.putText(frame, f'Distance Moved: {pixel_distance:.2f} px', (int(x1), int(y2) + 40), # Distance moved since last calculation in pixel
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 200, 255), 2)
                    cv2.putText(frame, f'Distance Moved: {real_world_distance:.2f} m', (int(x1), int(y2) + 60), # Distance moved since last calculation in meter
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 200, 255), 2)
                    cv2.putText(frame, f'Distance/Pixel: {distance_per_pixel:.2f} m', (int(x1), int(y2) + 80), # Distance per pixel in meter
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 200, 255), 2)

    previous_positions = current_positions

    cv2.polylines(frame,[np.array(Area_of_Interest,np.int32)],True,(0,255,255),2)

    # Display the frame
    cv2.imshow('frame',frame)

    key = cv2.waitKey(33) & 0xFF

    if key == 27:
        break
    elif key == ord('s') or key == ord('S'):  # Save frame when 'S' is pressed
        cv2.imwrite('captured_frame.jpg', frame)
        print('Frame captured and saved as captured_frame.jpg')

cap.release()
cv2.destroyAllWindows()

Resized Frame Size: Width = 960, Height = 540



0: 384x640 8 cars, 1 truck, 240.1ms
Speed: 8.0ms preprocess, 240.1ms inference, 13.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 132.9ms
Speed: 3.1ms preprocess, 132.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



Resized Frame Size: Width = 960, Height = 540
Resized Frame Size: Width = 960, Height = 540


0: 384x640 8 cars, 1 truck, 142.9ms
Speed: 4.1ms preprocess, 142.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 166.8ms
Speed: 4.0ms preprocess, 166.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


Resized Frame Size: Width = 960, Height = 540


...

Resized Frame Size: Width = 960, Height = 540



0: 384x640 11 cars, 1 truck, 131.1ms
Speed: 3.2ms preprocess, 131.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


Resized Frame Size: Width = 960, Height = 540



0: 384x640 11 cars, 1 truck, 132.3ms
Speed: 3.0ms preprocess, 132.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Resized Frame Size: Width = 960, Height = 540
